In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [13]:
import numpy as np
import tensorflow as tf
import datetime

from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.datasets import cifar10


In [3]:
# Clear any logs from previous runs
%rm -rf ./logs/

UsageError: Line magic function `%rm` not found.


In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Get the dataset and insert it into relevent variables

In [5]:
#Download the dataset
def download():
    (trainX, trainy), (testX, testy) = cifar10.load_data()
    print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
    print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

    #one hot encode target values
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    
    return trainX, trainy, testX, testy


In [6]:
#preprocess the dataset in it's relevent way
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    #normalize
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

In [15]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation=LeakyReLU(), input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation=LeakyReLU()))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation=LeakyReLU()))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = Adam(lr=0.0001)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [101]:
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = download()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()

	log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

	# fit model
	model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0, callbacks=[tensorboard_callback])
	# evaluate model
	_, acc = model.evaluate(testX, testY)
	print('> %.3f' % (acc * 100.0))
	# learning curves

In [17]:
run_test_harness()

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)
313/313 [==============================] - 2s 6ms/step - loss: 2.6882 - accuracy: 0.6474
> 64.740
